In [195]:
# !pip install pyarrow

In [196]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

import xgbtune 

In [197]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
0,2004-08-10,0.542679,0.563214,0.541964,0.562857,0.475869,351036000,NaN,NaN,0.021250,...,NaN,NaN,NaN,NaN,NaN,1,10,8,2004,33
1,2004-08-11,0.555357,0.555893,0.540357,0.553750,0.468169,322392000,NaN,NaN,0.015536,...,NaN,NaN,NaN,NaN,NaN,2,11,8,2004,33
2,2004-08-12,0.543750,0.550893,0.540714,0.542321,0.458507,226200800,NaN,NaN,0.010179,...,NaN,NaN,NaN,NaN,NaN,3,12,8,2004,33
3,2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465603,328048000,NaN,NaN,0.015714,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
4,2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464697,435674400,NaN,NaN,0.019286,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000,2024-06-21,210.389999,211.889999,207.110001,207.490005,207.490005,246421400,3.274162,0.032110,4.779999,...,0.012392,0.009259,-0.007538,0.010014,0.021213,4,21,6,2024,25
5001,2024-06-24,207.720001,212.699997,206.589996,208.139999,208.139999,80727000,3.174396,0.019971,6.110001,...,-0.019148,0.001649,-0.021058,-0.008235,-0.010559,0,24,6,2024,26
5002,2024-06-25,209.149994,211.380005,208.610001,209.070007,209.070007,56713900,3.384621,0.013439,2.770004,...,0.072649,0.007821,0.016588,0.017645,-0.008456,1,25,6,2024,26
5003,2024-06-26,211.500000,214.860001,210.639999,213.250000,213.250000,66213200,3.319522,0.014561,4.220001,...,0.028578,-0.007097,0.000053,0.006173,-0.006999,2,26,6,2024,26


In [198]:
# target = "target_close"
target = "close_percent_change_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,close_percent_change_1
date,
2004-08-10,-0.016180
2004-08-11,-0.020639
2004-08-12,0.015476
2004-08-13,-0.001945
2004-08-16,0.002924
...,...
2024-06-21,0.003133
2024-06-24,0.004468
2024-06-25,0.019993


In [199]:
x

,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,high_close_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2004-08-10,0.542679,0.563214,0.541964,0.562857,0.475869,351036000,NaN,NaN,0.021250,NaN,...,NaN,NaN,NaN,NaN,NaN,1,10,8,2004,33
2004-08-11,0.555357,0.555893,0.540357,0.553750,0.468169,322392000,NaN,NaN,0.015536,0.006964,...,NaN,NaN,NaN,NaN,NaN,2,11,8,2004,33
2004-08-12,0.543750,0.550893,0.540714,0.542321,0.458507,226200800,NaN,NaN,0.010179,0.002857,...,NaN,NaN,NaN,NaN,NaN,3,12,8,2004,33
2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465603,328048000,NaN,NaN,0.015714,0.016250,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464697,435674400,NaN,NaN,0.019286,0.015715,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-21,210.389999,211.889999,207.110001,207.490005,207.490005,246421400,3.274162,0.032110,4.779999,2.210007,...,0.012392,0.009259,-0.007538,0.010014,0.021213,4,21,6,2024,25
2024-06-24,207.720001,212.699997,206.589996,208.139999,208.139999,80727000,3.174396,0.019971,6.110001,5.209991,...,-0.019148,0.001649,-0.021058,-0.008235,-0.010559,0,24,6,2024,26
2024-06-25,209.149994,211.380005,208.610001,209.070007,209.070007,56713900,3.384621,0.013439,2.770004,3.240005,...,0.072649,0.007821,0.016588,0.017645,-0.008456,1,25,6,2024,26


In [200]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,volume,int64
0,on_balance_volume_1,int64
0,dividends_event_name,object
0,splits_event_name,object


In [201]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [202]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [203]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [204]:
# y_test

In [205]:
x_train

,open,high,low,close,adj close,volume,volitility_open_7,volitility_pct_change_7_open,high_low_7,high_close_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2015-05-14,31.852501,32.237499,31.790001,32.237499,29.015589,180814000.0,0.238384,0.012464,0.447498,0.734999,...,0.030364,0.018192,-0.004811,0.008612,-0.002098,3,14,5,2015,20
2023-02-03,148.029999,157.380005,147.830002,154.500000,153.237595,154357300.0,2.499204,0.016259,9.550003,6.559998,...,0.023500,0.008756,-0.010605,0.023809,0.004175,4,3,2,2023,5
2016-04-04,27.605000,28.047501,27.567499,27.780001,25.361149,149424800.0,0.552519,0.016878,0.480001,0.550001,...,-0.000094,0.020094,0.014877,0.008746,-0.042205,0,4,4,2016,14
2015-05-07,31.192499,31.520000,31.004999,31.315001,28.185286,175763600.0,0.487577,0.018114,0.515001,0.267500,...,0.004704,0.022846,0.007643,0.006970,0.023439,3,7,5,2015,19
2021-10-29,147.220001,149.940002,146.410004,149.800003,147.503387,124953200.0,0.884379,0.008136,3.529999,2.630005,...,0.011806,-0.009103,0.008127,-0.021361,-0.006256,4,29,10,2021,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-04,20.196428,20.328215,20.029285,20.178572,17.643908,377809600.0,0.630841,0.009558,0.298929,0.102501,...,-0.008758,0.014463,-0.002468,0.009791,-0.054444,2,4,12,2013,49
2013-06-11,15.562143,15.812857,15.475714,15.628571,13.497003,286112400.0,0.234132,0.014714,0.337143,0.138214,...,0.007951,0.003844,0.003907,0.029434,0.027183,1,11,6,2013,24
2004-09-21,0.674107,0.694107,0.668929,0.678750,0.573851,386652000.0,0.017367,0.018320,0.025178,0.020714,...,0.016499,0.039721,0.009091,NaN,NaN,1,21,9,2004,39


In [206]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [207]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = xgb.XGBRegressor(n_estimators=50, max_depth=3, eta=0.15, subsample=1, colsample_bytree=.4)
model.fit(x_train, y_train)

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

tune_params = {
    # "n_estimators": [100, 150],
    # "max_depth": [5, 6,],
    # "learning_rate": [0.05, 0.1],

    "n_estimators": [400, 450, 500, 550, 600],
    "max_depth": [6, 7, 8, 9, 11],
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4,],
    "subsample": [0.7, 0.8, 0.9, 1.0, 1.1],
    "colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5],

}

# {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.3}
# {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.2}

# xgb_model = xgb.XGBRegressor()
# # model = GridSearchCV(estimator=xgb_model, param_grid=tune_params)
# model = HalvingRandomSearchCV(estimator=xgb_model, param_distributions=tune_params)
# model.fit(x_train, y_train)

# print(model.best_params_)

pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [208]:
# print(model.best_params_)
# model


In [209]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

,close_percent_change_1,predicted
date,,
2022-06-16,0.011533,0.015244
2023-12-11,0.007920,0.001645
2007-07-16,0.005865,-0.006360
2022-09-09,0.038508,-0.005936
2017-06-12,0.008046,0.002415
...,...,...
2009-05-15,0.034553,0.011962
2023-09-21,0.004945,-0.000460
2010-06-28,-0.045211,-0.006410


In [210]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

0.020618194077830252

In [211]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

0.014276476473895652

In [212]:
# 0.01455269590461311
# 0.013673832772506729
# 0.012685448862479572

In [213]:
# rmsle
rmsle = np.sqrt(np.mean(np.log1p(np.abs(y_test["predicted"] - y_test[target])) ** 2))
rmsle

0.02010464002070757

In [214]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

inf

In [215]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")